In [1]:
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path):
    document = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(document)):
        page = document.load_page(page_num)
        text += page.get_text()
    return text

pdf_text = extract_text_from_pdf("policy-booklet-0923.pdf")


In [2]:
import pandas as pd

data = {
    "query": [
        "What is the definition of 'insured vehicle'?",
        "What does the policy cover for accidental damage?",
        "Are accessories covered under the policy?",
        "Does the policy cover personal belongings?",
        "What is the excess for accidental damage?",
        "What happens if I drive another car?",
        "Is windscreen damage covered?",
        "What is the procedure to make a claim?",
        "Are courtesy cars provided during repairs?",
        "What is the policy duration?",
        "How can I cancel my policy?",
        "What is the limit for medical expenses cover?",
        "Does the policy cover damage from natural disasters?",
        "What is covered under third-party liability?",
        "Is there any cover for legal expenses?",
        "What are the conditions for a no-claims discount?",
        "Can I drive in the EU with this policy?",
        "What is covered under breakdown assistance?",
        "Are there any exclusions to the policy?",
        "What is the process for adding a driver to the policy?",
        "What happens if my car is stolen?",
        "Does the policy cover modifications to the vehicle?",
        "What is the limit for personal accident cover?",
        "How do I report an accident?",
        "What are the benefits of using an approved repairer?",
        "Is there a mileage limit for this policy?",
        "What types of vehicles are covered?",
        "What is the grace period for policy renewal?",
        "Are trailers covered under this policy?",
        "What should I do if my personal details change?"
    ],
    "response": [
        "The vehicle described in the current certificate of motor insurance.",
        "We will pay for damage to the insured vehicle caused accidentally.",
        "Yes, accessories are covered as long as they are in or on the vehicle.",
        "Yes, we cover personal belongings up to £250.",
        "The excess for accidental damage is £100.",
        "You are covered third party only if the other car is not owned by you or hired to you.",
        "Yes, windscreen damage is covered with an excess of £75.",
        "To make a claim, call our 24-hour claims line on the number provided in your policy document.",
        "Yes, a courtesy car is provided if your car is being repaired by one of our approved repairers.",
        "The policy is effective for one year from the start date shown in the certificate of insurance.",
        "You can cancel your policy by providing us with written notice.",
        "We will pay up to £100 for each person for medical expenses incurred.",
        "Yes, the policy covers damage caused by natural disasters such as floods or storms.",
        "The policy covers your legal liability for death or injury to others and damage to their property.",
        "Legal expenses cover is not included as standard but can be added as an optional extra.",
        "A no-claims discount is applied if you do not make any claims during the policy year.",
        "Yes, the policy provides the minimum legal cover required to drive in the EU.",
        "Breakdown assistance includes roadside repair and recovery to the nearest garage.",
        "Yes, exclusions include driving under the influence of alcohol or drugs, and using the vehicle for racing.",
        "To add a driver, you must contact us with their details, and an additional premium may be required.",
        "If your car is stolen, we will pay for its replacement or the market value of the vehicle.",
        "Modifications are covered only if they are declared and accepted by us.",
        "We provide up to £5,000 for death or serious injury caused by a car accident.",
        "Report the accident by calling our claims line and providing all necessary details.",
        "Using an approved repairer ensures quality repairs and a courtesy car while your vehicle is being fixed.",
        "No, there is no mileage limit imposed on the policy.",
        "The policy covers private cars used for social, domestic, and pleasure purposes.",
        "There is no formal grace period; the policy must be renewed before the expiry date.",
        "Trailers are covered while attached to the insured vehicle, but there is no cover for their contents.",
        "Notify us immediately if any of your personal details change to ensure your policy remains valid."
    ]
}

df = pd.DataFrame(data)
df.to_csv("queries_responses.csv", index=False)


In [3]:
from transformers import RagTokenizer, RagRetriever, RagSequenceForGeneration
from transformers import TrainingArguments, Trainer
from datasets import Dataset
import pandas as pd

# Load the dataset
data = pd.read_csv(r"C:\Users\Sanskruti\Desktop\Workspace2\queries_responses.csv")
dataset = Dataset.from_pandas(data)

In [4]:
import faiss
from sentence_transformers import SentenceTransformer

# Load the dataset
dataset_df = pd.read_csv(r"C:\Users\Sanskruti\Desktop\Workspace2\queries_responses.csv")

# Load a pre-trained Sentence Transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Encode the text data
embeddings = model.encode(dataset_df['response'].tolist(), convert_to_tensor=True)

# Create FAISS index
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings.cpu().detach().numpy())

# Save the FAISS index
faiss.write_index(index, "faiss_index.bin")


c:\Users\Sanskruti\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
import faiss
from sentence_transformers import SentenceTransformer

# Load the dataset
dataset_df = pd.read_csv(r"C:\Users\Sanskruti\Desktop\Workspace2\queries_responses.csv")

# Load a pre-trained Sentence Transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Encode the responses
embeddings = model.encode(dataset_df['response'].tolist(), convert_to_tensor=True)

# Convert embeddings to numpy array
embeddings_np = embeddings.cpu().detach().numpy()

# Create FAISS index
dimension = embeddings_np.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings_np)

# Save the FAISS index
faiss.write_index(index, "faiss_index.bin")

print("FAISS index saved as faiss_index.bin")


c:\Users\Sanskruti\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


FAISS index saved as faiss_index.bin


In [6]:
from datasets import load_dataset, Dataset

# Load the dataset using Hugging Face's datasets library
dataset = Dataset.from_pandas(dataset_df)

# Save the dataset in the appropriate format for RAG
dataset.save_to_disk("query_response_dataset")

print("Dataset saved in Hugging Face format")


Saving the dataset (0/1 shards):   0%|          | 0/30 [00:00<?, ? examples/s]

Dataset saved in Hugging Face format


In [7]:
import pandas as pd
from datasets import Dataset

# Load your CSV
df = pd.read_csv("queries_responses.csv")

# Create the required columns
df = df.rename(columns={"query": "title", "response": "text"})
df["embeddings"] = None  # Placeholder for embeddings

# Convert to Huggingface Dataset
dataset = Dataset.from_pandas(df)


In [8]:
import pandas as pd
from datasets import Dataset
from sentence_transformers import SentenceTransformer

# Load your CSV
df = pd.read_csv(r"C:\Users\Sanskruti\Desktop\Workspace2\queries_responses.csv")

# Create the required columns
df = df.rename(columns={"query": "title", "response": "text"})
df["embeddings"] = None  # Placeholder for embeddings

# Convert to Huggingface Dataset
dataset = Dataset.from_pandas(df)

# Load a pre-trained sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings for the text column
embeddings = model.encode(df['text'].tolist(), convert_to_numpy=True)

# Assign embeddings back to the dataframe
df["embeddings"] = list(embeddings)  # Convert numpy arrays to lists

# Convert back to Huggingface Dataset
dataset = Dataset.from_pandas(df)


c:\Users\Sanskruti\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [9]:
import numpy as np
# Save the dataset to disk
dataset_path = "./dataset"
index_path = "./index"
dataset.save_to_disk(dataset_path)

# Save the FAISS index
import faiss

# Ensure embeddings are a 2D numpy array
embeddings_2d = np.vstack(dataset["embeddings"]) # type: ignore

# Create and train the FAISS index
faiss_index = faiss.IndexFlatL2(embeddings_2d.shape[1])
faiss_index.add(embeddings_2d)

# Save the FAISS index to disk
faiss.write_index(faiss_index, index_path)


Saving the dataset (0/1 shards):   0%|          | 0/30 [00:00<?, ? examples/s]

In [10]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Step 1: Load and Prepare Data
def load_data(file_path):
    data = pd.read_csv(r"C:\Users\Sanskruti\Desktop\Workspace2\queries_responses.csv")
    return data

# Step 2: Train Model
def train_model(data):
    model = SentenceTransformer('all-MiniLM-L6-v2')
    embeddings = model.encode(data['response'].tolist(), convert_to_tensor=True)
    return model, embeddings

In [11]:
# Step 2: Train Model
def train_model(data):
    model = SentenceTransformer('all-MiniLM-L6-v2')
    embeddings = model.encode(data['response'].tolist(), convert_to_tensor=True)
    return model, embeddings

In [12]:
# Step 3: Create FAISS Index
def create_faiss_index(embeddings):
    embeddings_np = embeddings.cpu().detach().numpy()
    dimension = embeddings_np.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings_np)
    faiss.write_index(index, "faiss_index.bin")
    return index

In [13]:
# Step 4: Query Handling
def handle_query(query, model, index, data):
    query_embedding = model.encode([query], convert_to_tensor=True)
    query_embedding_np = query_embedding.cpu().detach().numpy()
    D, I = index.search(query_embedding_np, k=1)
    return data['response'].iloc[I[0][0]]

In [14]:
# Step 5: Evaluation and Improvement
def evaluate_model(test_data, model, index, data):
    correct = 0
    for _, row in test_data.iterrows():
        response = handle_query(row['query'], model, index, data)
        if response == row['response']:
            correct += 1
    accuracy = correct / len(test_data)
    return accuracy

In [15]:
# Main script to execute the steps
data = load_data('queries_responses.csv')
model, embeddings = train_model(data)
index = create_faiss_index(embeddings)

# Example of handling a query
query = "What is the definition of 'insured vehicle'?"
response = handle_query(query, model, index, data)
print(f"Response: {response}")

c:\Users\Sanskruti\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Response: The vehicle described in the current certificate of motor insurance.


In [16]:
# Evaluate model
test_data = load_data('test_queries_responses.csv')
accuracy = evaluate_model(test_data, model, index, data)
print(f"Model Accuracy: {accuracy:.2f}")

Model Accuracy: 0.77


In [18]:
import pandas as pd
from datasets import Dataset
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Load your CSV using a raw string for the file path
df = pd.read_csv(r"C:\Users\Sanskruti\Desktop\Workspace2\queries_responses.csv")

# Create the required columns
df = df.rename(columns={"query": "title", "response": "text"})
df["embeddings"] = None  # Placeholder for embeddings

# Convert to Huggingface Dataset
dataset = Dataset.from_pandas(df)

# Load a pre-trained sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings for the text column
embeddings = model.encode(df['text'].tolist(), convert_to_numpy=True)

# Assign embeddings back to the dataframe
df["embeddings"] = list(embeddings)  # Convert numpy arrays to lists

# Convert back to Huggingface Dataset
dataset = Dataset.from_pandas(df)

# Save the dataset to disk
dataset.save_to_disk("./dataset")

# Ensure embeddings are a 2D numpy array
embeddings_2d = np.vstack(df["embeddings"])

# Create and train the FAISS index
faiss_index = faiss.IndexFlatL2(embeddings_2d.shape[1])
faiss_index.add(embeddings_2d)

# Save the FAISS index to disk
faiss.write_index(faiss_index, "./index")


c:\Users\Sanskruti\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Saving the dataset (0/1 shards):   0%|          | 0/30 [00:00<?, ? examples/s]